# Google Data Analytics Course 8 - Capstone
**Author: Carlos Ramos**

**Date: 15/10/2022**

#### **Case Study 1: How Does a Bike-Share Navigate Speedy Success?**


##### **Background**

This is the analysis performed for the Case 1 from Google Data Analytics Certificate called **Cyclistic bike-share analysis**


##### **Business Purpose**

The analysis is focused to answer the Business question: __How do annual members and casual riders use Cyclistic bikes differently?__

<u> *Information about the dataset* </u>

Dataset policy:

* The data has been made available by MotivateInternational Inc. under [this license](https://www.divvybikes.com/data-license-agreement)
* It is external data stored in the cloud
* It used the long data time format
* First-party data (reliable and original)
* There is a file for each past 2022 year and availability months from 2023
* There is not Personally Identifiable Information (PII)

Dataset Process:

* Each csv file is Downloaded 
* Each csv file is read in R and uploaded to a MS SQL Server 2019 database
* All data is appended to the Database's Table _trips_data_
* A total of 8.825.826 rows uploaded
* Used in Tableau Public destop tool to create the visuals and perform analysis


##### **Data Analysis**

**Install required Packages**

In [ ]:
install.packages("tidyverse")
install.packages("RODBC", type = "source", configure.args = c("--with-odbc-include=/usr/local/include/", "--with-odbc-lib=/usr/local/lib/")) # nolint: line_length_linter.

In [ ]:
library(tidyverse)
library(RODBC)

Load the CSV data from folder into data frames

In [4]:
df_tryps_202201 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202201-divvy-tripdata.csv") # nolint
df_tryps_202202 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202202-divvy-tripdata.csv") # nolint
df_tryps_202203 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202203-divvy-tripdata.csv") # nolint
df_tryps_202204 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202204-divvy-tripdata.csv") # nolint
df_tryps_202205 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202205-divvy-tripdata.csv") # nolint
df_tryps_202206 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202206-divvy-tripdata.csv") # nolint
df_tryps_202207 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202207-divvy-tripdata.csv") # nolint
df_tryps_202208 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202208-divvy-tripdata.csv") # nolint
df_tryps_202209 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202209-divvy-tripdata.csv") # nolint
df_tryps_202210 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202210-divvy-tripdata.csv") # nolint
df_tryps_202211 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202211-divvy-tripdata.csv") # nolint
df_tryps_202212 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202212-divvy-tripdata.csv") # nolint

df_tryps_202301 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202301-divvy-tripdata.csv") # nolint
df_tryps_202302 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202302-divvy-tripdata.csv") # nolint
df_tryps_202303 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202303-divvy-tripdata.csv") # nolint
df_tryps_202304 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202304-divvy-tripdata.csv") # nolint
df_tryps_202305 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202305-divvy-tripdata.csv") # nolint
df_tryps_202306 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202306-divvy-tripdata.csv") # nolint
df_tryps_202307 <- read.csv("/Users/carlos/Desktop/Python/R/capstone/data/202307-divvy-tripdata.csv") # nolint

Function for converting started_at and ended_at columns to datatime type

In [5]:
# Function to covert to datatime type two columns in the DF
convert_datetime <- function(data) {
    data$started_at <- strptime(data$started_at,"%Y-%m-%d %H:%M:%S")
    data$ended_at <- strptime(data$ended_at,"%Y-%m-%d %H:%M:%S")
}

Conversion of all Data Frames

In [7]:
# Year 2022
convert_datetime(df_tryps_202201)
convert_datetime(df_tryps_202202)
convert_datetime(df_tryps_202203)
convert_datetime(df_tryps_202204)
convert_datetime(df_tryps_202205)
convert_datetime(df_tryps_202206)
convert_datetime(df_tryps_202207)
convert_datetime(df_tryps_202208)
convert_datetime(df_tryps_202209)
convert_datetime(df_tryps_202210)
convert_datetime(df_tryps_202211)
convert_datetime(df_tryps_202212)
# Year 2023
convert_datetime(df_tryps_202301)
convert_datetime(df_tryps_202302)
convert_datetime(df_tryps_202303)
convert_datetime(df_tryps_202304)
convert_datetime(df_tryps_202305)
convert_datetime(df_tryps_202306)
convert_datetime(df_tryps_202307)

Upload the data to MS SQL Server
* Create a Connection (RODBC package used)
* Function is created to upload the DF to the database 
* Function is called for every DF from 2022 to 2023-07 (available data in 2023)

In [ ]:
# Create connection to the DB
con <- RODBC::odbcDriverConnect(connection = "driver=ODBC driver 18 for SQL Server;server=192.168.1.36;TrustServerCertificate=yes;database=Google;UID=esscrao;PWD=****") # nolint

# Function to upload DF to the DB
# Params: data = DF , conn = database connection)
upload_data <- function(data, conn){
    ColumnsOfTable       <- sqlColumns(con, "trips_data") # nolint
    varTypes             <- as.character(ColumnsOfTable$TYPE_NAME) # nolint
    names(varTypes)      <- as.character(ColumnsOfTable$COLUMN_NAME)
    colnames(data) <- as.character(ColumnsOfTable$COLUMN_NAME)
    odbcSetAutoCommit(conn, autoCommit = FALSE)
    sqlSave(conn, data, tablename = "trip_data", fast = FALSE, colnames = FALSE, rownames = FALSE, append = TRUE, verbose = FALSE, safer = TRUE) # nolint
    odbcSetAutoCommit(conn, autoCommit = TRUE)
    return (nrow(data)) # nolint
}


In [ ]:
# 2022
upload_data(df_tryps_202201, con)
upload_data(df_tryps_202202, con)
upload_data(df_tryps_202203, con)
upload_data(df_tryps_202204, con)
upload_data(df_tryps_202205, con)
upload_data(df_tryps_202206, con)
upload_data(df_tryps_202207, con)
upload_data(df_tryps_202208, con)
upload_data(df_tryps_202209, con)
upload_data(df_tryps_202210, con)
upload_data(df_tryps_202211, con)
upload_data(df_tryps_202212, con)
#2023
upload_data(df_tryps_202301, con)
upload_data(df_tryps_202302, con)
upload_data(df_tryps_202303, con)
upload_data(df_tryps_202304, con)
upload_data(df_tryps_202305, con)
upload_data(df_tryps_202306, con)
upload_data(df_tryps_202307, con)
upload_data(df_tryps_202308, con)
upload_data(df_tryps_202309, con)
upload_data(df_tryps_202310, con)
upload_data(df_tryps_202311, con)
upload_data(df_tryps_202312, con)


##### **Tableau Proccess**

* Create a live connection to the MS SQL Server table with the data using custom SQL to calculate in DB the time per trip
* Create some calculations in Tableau:
    - Trip = string with Origin to Destination
    - Origin = MAKEPOINT([start_lat],[start_lng])
    - Destination = MAKEPOINT([end_lat],[end_lng])
    - Distance = DISTANCE([Origin],[Destination],'km')
* Create different graphs to understand the data
* Create a Dashboard with the result
* Published to Tableau Public [Dashboard](https://public.tableau.com/app/profile/carlos.ramos2030/viz/CapstoneGoogle_16971918360790/Dashboard1)

### **KEY TAKEAWAYS AND CONCLUSIONS**

- Monthly trips per Months are increasing in 2023 compared to 2022 for Members but decreasing in Casuals
- Members' trips are always bigger than Casuals' 
- On weekends casual bike riders increases. The most popular day for bike renting for all subscribers was Sunday.
- The months with more trips are usually from June to Septembre
- The busy hours in the service are from 16:00 to 18:00, being around 40% more from Members than casual at 17h
- Casual Trips have more AVG duraction than the Members
- Casual Trips have a little bit more AVG distances than members
- Members usage is 60.56% compared to Casuals' 39.44%
- Members usage more or less the same Electric and Classic Bikes (and don't use docked bikes)
- Casuals usage more the Electric bikes (54.58%) compared to Classic Bikes (38.53%) and docked Bikes (6.89%)
- Distances are similar along the year per bike type and minutes of use
- Electric bikes are more used but with less time
- Docked bikes are used in AGV time more than others, being distances little lower than Electrics' ones
- Trips from Ellis Ave & 60th St to University Ave & 57th St are the most frecuently used
- The data shows a clear seasonal fluctuations. It peaked during the summer months and was at its lowest point from the late fall to early spring.
- Curiosity on the avg time of trips from Casual to members: Members take less time to perform similar trips than Casuals

_See the Tableau [Dashboard](https://public.tableau.com/app/profile/carlos.ramos2030/viz/CapstoneGoogle_16971918360790/Dashboard1)_

### **Recommendations**

- Investing on Electric bikes is a good suggestion, since they are the most used type of bikes
- Having the contract option of weekend Members may increase the number of Members in total
- Moving bikes from low usage to high usage stations could also increase the service usage